<a href="https://colab.research.google.com/github/HarshitaDua07/AI-wellness-assisstent-flow-DEMO/blob/main/AI_wellness_assisstent_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok


In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional, List, Dict
import re
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI(title="AI Wellness Demo", version="0.1")

class MessageIn(BaseModel):
    text: str
    user_id: Optional[str] = None

SYMPTOM_KEYWORDS = {
    "tired": ["Anemia", "Chronic Fatigue Syndrome", "Depression"],
    "fatigue": ["Anemia", "Chronic Fatigue Syndrome", "Hypothyroidism"],
    "dizzy": ["Hypotension", "Dehydration", "Vertigo"],
    "faint": ["Syncope", "Severe Dehydration"],
    "headache": ["Migraine", "Tension Headache", "Hypertension"],
    "fever": ["Infection", "Influenza"],
    "chest pain": ["Heart attack", "Angina"],
    "shortness of breath": ["Asthma", "Pneumonia", "Heart failure"]
}

URGENCY_PHRASES = ["chest pain", "shortness of breath", "faint", "passing out", "severe bleeding"]

RECOMMENDATIONS = {
    "Anemia": "Eat iron-rich foods (spinach, lentils, beetroot). Consider a blood test.",
    "Chronic Fatigue Syndrome": "Maintain good sleep hygiene and gentle exercise.",
    "Hypotension": "Hydrate, rise slowly from sitting.",
    "Dehydration": "Drink extra fluids, avoid alcohol/caffeine.",
    "Vertigo": "Try vestibular exercises; consult ENT if persistent.",
    "Migraine": "Rest in a dark room, use OTC meds if safe.",
    "Heart attack": "Possible emergency. Call emergency services immediately."
}

def detect_intent(text: str) -> str:
    if any(k in text.lower() for k in SYMPTOM_KEYWORDS.keys()):
        return "SymptomCheck"
    return "GeneralQuery"

def extract_entities(text: str) -> List[str]:
    txt = text.lower()
    found = []
    keys = sorted(SYMPTOM_KEYWORDS.keys(), key=lambda x: -len(x))
    for k in keys:
        if k in txt:
            found.append(k)
    words = re.findall(r"\w+", txt)
    for w in words:
        if w in SYMPTOM_KEYWORDS and w not in found:
            found.append(w)
    return found

def rank_conditions(symptoms: List[str]) -> List[Dict]:
    scores = {}
    for s in symptoms:
        for cond in SYMPTOM_KEYWORDS.get(s, []):
            scores[cond] = scores.get(cond, 0) + 1
    if not scores:
        return []
    max_score = max(scores.values())
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [{"condition": c, "score": round(s / max_score, 2)} for c, s in ranked]

def check_urgency(text: str) -> bool:
    return any(p in text.lower() for p in URGENCY_PHRASES)

@app.post("/api/v1/message")
def message(in_msg: MessageIn):
    intent = detect_intent(in_msg.text)
    entities = extract_entities(in_msg.text)
    conditions = rank_conditions(entities)
    urgent = check_urgency(in_msg.text)
    recommendations = {
        c["condition"]: RECOMMENDATIONS.get(
            c["condition"], "General advice: stay hydrated and consult a clinician."
        ) for c in conditions
    }
    return {
        "intent": intent,
        "entities": entities,
        "conditions": conditions,
        "recommendations": recommendations,
        "safety": {"urgent": urgent},
        "disclaimer": "This is not a diagnosis. Consult a doctor."
    }


In [12]:
# Start server in background
!pip install fastapi uvicorn pyngrok nest_asyncio requests

import nest_asyncio
from pyngrok import ngrok

nest_asyncio.apply()

from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional, List, Dict

app = FastAPI()

class MessageIn(BaseModel):
    text: str
    user_id: Optional[str] = None

SYMPTOM_KEYWORDS = {
    "tired": ["Anemia", "Chronic Fatigue Syndrome", "Depression"],
    "fatigue": ["Anemia", "Chronic Fatigue Syndrome", "Hypothyroidism"],
    "dizzy": ["Hypotension", "Dehydration", "Vertigo"],
}

def extract_entities(text: str) -> List[str]:
    return [k for k in SYMPTOM_KEYWORDS if k in text.lower()]

def rank_conditions(symptoms: List[str]) -> List[Dict]:
    scores = {}
    for s in symptoms:
        for cond in SYMPTOM_KEYWORDS.get(s, []):
            scores[cond] = scores.get(cond, 0) + 1
    return [{"condition": c, "score": s} for c, s in scores.items()]

@app.post("/api/v1/message")
def message(in_msg: MessageIn):
    entities = extract_entities(in_msg.text)
    conditions = rank_conditions(entities)
    return {"entities": entities, "conditions": conditions}

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run uvicorn in background
import threading
import uvicorn
threading.Thread(target=uvicorn.run, args=(app,), kwargs={"host":"0.0.0.0", "port":8000}).start()


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-19' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

Public URL: NgrokTunnel: "https://a25e5bc61904.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [592]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [14]:
import requests

url = "https://a25e5bc61904.ngrok-free.app/api/v1/message"

# Ask the user for input
user_message = input("Enter your message: ")

# Create payload dynamically
payload = {"text": user_message, "user_id": "demo_user"}

# Send POST request
r = requests.post(url, json=payload)

# Print the JSON response
print(r.json())


Enter your message: I have been feeling tired and dizzy.
INFO:     35.231.53.59:0 - "POST /api/v1/message HTTP/1.1" 200 OK
{'entities': ['tired', 'dizzy'], 'conditions': [{'condition': 'Anemia', 'score': 1}, {'condition': 'Chronic Fatigue Syndrome', 'score': 1}, {'condition': 'Depression', 'score': 1}, {'condition': 'Hypotension', 'score': 1}, {'condition': 'Dehydration', 'score': 1}, {'condition': 'Vertigo', 'score': 1}]}
